In [1]:
import duckdb

In [3]:
conn = duckdb.connect('c:/Users/Fgllc/OneDrive/Escritorio/USC/etl_project/cinema_tickets.duckdb')
conn

THE POINT OF THE NOTEBOOK IS GOING TO BE TO LOOK AT THE ISSUES FROM THE VARIABLES "total_sales" , "ticket_price"

In [48]:
query = """
SELECT *
FROM main.cinema_tickets
LIMIT 5
"""

In [14]:
conn.execute(query).fetchdf()

,film_code,cinema_code,total_sales,tickets_sold,tickets_out,show_time,occu_perc,ticket_price,ticket_use,capacity,date,month,quarter,day
0,1492,304,3900000,26,0,4,4.26,150000.0,26,610.328638,2018-05-05,5,2,5
1,1492,352,3360000,42,0,5,8.08,80000.0,42,519.801980,2018-05-05,5,2,5
2,1492,489,2560000,32,0,4,20.00,80000.0,32,160.000000,2018-05-05,5,2,5
3,1492,429,1200000,12,0,1,11.01,100000.0,12,108.991826,2018-05-05,5,2,5
4,1492,524,1200000,15,0,3,16.67,80000.0,15,89.982004,2018-05-05,5,2,5


"ticket_price" doesn't make any sense. A ticket price for a movie for $150,000? Maybe it is already multiplied by the number of tickets? Need to dive deeper.

In [46]:
#checking if the ticket_price is maybe multiplied by the tickets sold

query2 = """
SELECT
    total_sales,
    tickets_sold,
    ticket_price,
    ticket_price / tickets_sold AS unit_price
FROM main.cinema_tickets
LIMIT 10
"""

In [47]:
conn.execute(query2).fetchdf()
#some takeaways:
#those unit prices are waaaay to high. ticket price is not a unit price, it is obvious thanks to seeing this. 
#it is likely that "unit_price" 
#thanks to all this, when you multiply ticket_price*ticket_sold, thats why total_sales are this high and not making sense. 
# the main problem is that ticket_price might be the total sale (possibly misnamed), behaving more like the real "total_sales"
#technically total_sales and ticket_price are actually the same. pick one to stick with. 

,total_sales,tickets_sold,ticket_price,unit_price
0,3900000,26,150000.000000,5769.230769
1,3360000,42,80000.000000,1904.761905
2,2560000,32,80000.000000,2500.000000
3,1200000,12,100000.000000,8333.333333
4,1200000,15,80000.000000,5333.333333
5,1050000,7,150000.000000,21428.571429
6,1020000,10,102000.000000,10200.000000
7,750000,5,150000.000000,30000.000000
8,750000,11,68181.818182,6198.347107
9,600000,4,150000.000000,37500.000000


HERE IM GOING TO CREATE THE BASE MODEL WHERE TOTAL_SALES IS ELIMINATED SINCE IT DOESN'T MAKE SENSE. 

In [ ]:
query3 = """
SELECT
    film_code,
    cinema_code,
    ticket_price AS total_revenue,
    tickets_sold,
    tickets_out,
    show_time,
    occu_perc,
    ticket_use,
    capacity,
    date,
    month,
    quarter,
    day
FROM main.cinema_tickets
"""

In [67]:
conn.execute(query3).fetchdf()

,film_code,cinema_code,total_revenue,tickets_sold,tickets_out,show_time,occu_perc,ticket_use,capacity,date,month,quarter,day
0,1492,304,150000.0,26,0,4,4.26,26,610.328638,2018-05-05,5,2,5
1,1492,352,80000.0,42,0,5,8.08,42,519.801980,2018-05-05,5,2,5
2,1492,489,80000.0,32,0,4,20.00,32,160.000000,2018-05-05,5,2,5
3,1492,429,100000.0,12,0,1,11.01,12,108.991826,2018-05-05,5,2,5
4,1492,524,80000.0,15,0,3,16.67,15,89.982004,2018-05-05,5,2,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
142519,1569,495,60000.0,22,0,2,3.86,22,569.948187,2018-11-04,11,4,4
142520,1569,474,80000.0,15,0,1,65.22,15,22.999080,2018-11-04,11,4,4
142521,1569,524,132500.0,8,0,3,9.20,8,86.956522,2018-11-04,11,4,4
142522,1569,529,120000.0,5,0,2,5.00,5,100.000000,2018-11-04,11,4,4
